---
Supplement to Master Thesis Presented to the Department of Economics at the Rheinische Friedrich-Wilhelms-Universität Bonn | by
[Muhammad Danial Syed](https://github.com/MDanialSyed) <br>

---
# "CAUSAL INFERENCE USING RANDOM FORESTS AND LASSO" - Code
#### by Muhammad Danial Syed

This notebook contains replication codes for my thesis based on the Generalized Random Forest method introduced in [Generalized Random Forests](https://projecteuclid.org/journals/annals-of-statistics/volume-47/issue-2/Generalized-random-forests/10.1214/18-AOS1709.full) (Athey et al., 2019)

The following packages are used to implement the method and for comparison purposes with [Double Machine Learning](https://academic.oup.com/ectj/article/21/1/C1/5056401) and [Double Selection](https://academic.oup.com/restud/article-abstract/81/2/608/1523757?redirectedFrom=fulltext)
- [GRF](https://grf-labs.github.io/grf/index.html)
- [DoubleML](https://docs.doubleml.org/stable/index.html)
- [MLR3 Learners](https://mlr3learners.mlr-org.com)
- [hdm](https://cran.r-project.org/web/packages/hdm/index.html)

#### Source material
The project uses the above `GRF` package for its main computations and also relies on certain modifications of the [source code](https://github.com/grf-labs/grf/blob/master/r-package/grf/R/dgps.R) to alter the data generating process.

#### Overview

Part I: A multi-part simulation study that complements the theoretical portion of my thesis and critically examines some of the main components of GRF:
1. Flexible estimation of nonparametric functions.
2. Causal forest with and without Honesty.
3. Causal forest compared with Double Machine Learning and Post Double-Selection.
4. Predicting Treatment Heterogeneity with causal forests.

Part II: A two-pronged robustness check of the empirical applicability of GRF.
1. *The Impacts of Microcredit: Evidence from Bosnia and Herzegovina* (Augsburg et al., 2015)
2. *Effect of 401(k) eligibility and participation on net financial assets*

#### Table of Content

1. [Flexible estimation of nonparametric functions - OLS and GRF Fit (Figure 1)](#1)  <br>
    1.1. [Data Generating Process 1](#2) <br>
2. [Estimating Average Treatment Effects](#2) <br>
    2.1. [Data Generating Process 2](#2) <br>
    2.2. [Simulation - Honest and Dishonest Causal Forest for ATEs](#2) <br>
    2.3. [Causal Forests, Double Machine Learning, and Post Double Selection LASSO](#2) <br>
3. [Predicting HTEs](#8) <br>
    3.1. [Data Generating Process 3](#9) <br>
    3.2. [Simulation Experiments: Honest and Dishonest Forests for HTEs](#11) <br>
4. [Empirical Application: *The Impacts of Microcredit: Evidence from Bosnia and Herzegovina*](#15) <br>
5. [Empirical Application: *Effect of 401(k) eligibility and participation on net financial assets*](#16) <br>

---
## Part I: Simulation Studies
---

### Initialization 

In [ ]:
rm(list=ls())

suppressMessages(library(grf))
suppressMessages(library(hdm))
suppressMessages(library(doMC))
suppressMessages(library(mlr3))
suppressMessages(library(haven))
suppressMessages(library(ranger))
suppressMessages(library(glmnet))
suppressMessages(library(xtable))
suppressMessages(library(ggplot2))
suppressMessages(library(mvtnorm))
suppressMessages(library(reshape2)) 
suppressMessages(library(parallel))
suppressMessages(library(DoubleML))
suppressMessages(library(data.table))
suppressMessages(library(mlr3learners))
suppressMessages(library(clusterGeneration))
lgr::get_logger("mlr3")$set_threshold("warn")

numCores <- detectCores()
registerDoMC(cores = numCores)

___
### 1 - Flexible estimation of nonparametric functions - OLS and GRF Fit (Figure 1)

In [ ]:
set.seed(123)

#Generate data from DGP 1
X1 <- as.matrix(sort(rnorm(120,40,20)))
Y1 <- as.matrix(rnorm(120,90 + exp(0.06*X1), 1))

X0 <- as.matrix(sort(rnorm(120,20,20)))
Y0 <- as.matrix(rnorm(120,72+3*sqrt(abs(X0)),1))

# Data for True Response Curve
X0_response <- sort(rnorm(120000,20,10))
X1_response <- sort(rnorm(120000,40,10))

X.test <- matrix(seq(0, 60, length.out = 120))

# Train Regression Forest
forest.treat <- regression_forest(X1, Y1)
Y1.hat <- predict(forest.treat,X.test)$predictions

forest.untreat <- regression_forest(X0, Y0)
Y0.hat <- predict(forest.untreat,X.test)$predictions

# OLS Plot
plot(X0,Y0,xlim = c(2,60),ylim = c(75,130),pch = 5,xlab='X',ylab='Y',cex=1.2,cex.axis=1.5,cex.lab=1.5)
points(X1,Y1,xlim = c(0,60),ylim = c(75,130), cex=1.2)
lines(X0_response,72+3*sqrt(abs(X0_response)), col = 1, lwd = 1)
lines(X1_response,90 + exp(0.06*X1_response), col = 1, lwd = 1)
abline(lm(Y1 ~ X1), col = 4, lwd = 3)
abline(lm(Y0 ~ X0), col = 3, lwd = 3)

# GRF Plot
plot(X0,Y0,xlim = c(2,60),ylim = c(75,130),pch = 5,xlab='X',ylab='Y',cex=1.2,cex.axis=1.5,cex.lab=1.5)
points(X1,Y1,xlim = c(0,60),ylim = c(75,130), cex=1.2)
lines(X0_response,72+3*sqrt(abs(X0_response)), col = 1, lwd = 1)
lines(X1_response,90 + exp(0.06*X1_response), col = 1, lwd = 1)
lines(X.test, Y1.hat, col =2, lwd = 3,lty=77)
lines(X.test, Y0.hat, col =2, lwd = 3,lty=77)

___
### 2 - Estimating Average Treatment Effects

### 2.1. Data Generating Process 2
- Adapted from GRF source code.
- Modified to include constant treatment effect for select DGPs.

In [ ]:
generate_DGP <- function(n, p, sigma.m = 1, sigma.tau = 0.1, sigma.noise = 1,
                                 dgp = c("aw1","nw1","nw2","nw3")) {

  # Create data
    if (dgp == "aw1") {
    X <- matrix(runif(n * p, min = 0, max = 1), n, p)
    tau <- rep(1, n)  # Treatment effect is one
    e <- (1 / 4)  * (exp(X[,3]) /(1 + exp(X[,3])))  # Confounding
    W <- rbinom(n = n, size = 1, prob = e)
    m <- 2 * X[, 1] - 1 + e * tau
    V <- 1
  } else if (dgp == "nw1") {
    # Difficult nuisance components, easy treatment effect function.
    X <- matrix(runif(n * p), n, p)
    tau <- rep(1, n)
    eta <- 0.1
    e <- pmax(eta, pmin(sin(pi * X[, 1] * X[, 2]), 1 - eta))
    W <- rbinom(n = n, size = 1, prob = e)
    m <- sin(pi * X[, 1] * X[, 2]) + 2 * (X[, 3] - 0.5)^2 + X[, 4] + 0.5 * X[, 5] + e * tau
    V <- 1
  } else if (dgp == "nw2") {
    # Randomized trial
    X <- matrix(rnorm(n * p), n, p)
    tau <- rep(1, n)
    e <- rep(0.5, n)
    W <- rbinom(n = n, size = 1, prob = e)
    m <- pmax(0, X[, 1] + X[, 2], X[, 3]) + pmax(0, X[, 4] + X[, 5]) + e * tau
    V <- 1
  } else if (dgp == "nw3") {
    # Strong confounding
    X <- matrix(rnorm(n * p), n, p)
    tau <- rep(1, n)
    e <- 1 / (1 + exp(X[, 2] + X[, 3]))
    W <- rbinom(n = n, size = 1, prob = e)
    m <- 2 * log(1 + exp(X[, 1] + X[, 2] + X[, 3])) + e * tau
    V <- 1
  } 

  # Scale and return data
  if (!is.na(sd(m)) & !(sd(m) == 0)) {
    m <- m / sd(m) * sigma.m
  }
  if (!is.na(sd(tau)) & !(sd(tau) == 0)) {
    tau <- tau / sd(tau) * sigma.tau
  }
  V <- V / mean(V) * sigma.noise^2
  Y <- m + (W - e) * tau + sqrt(V) * rnorm(n)
    
    colnames(X) <- paste0("X", 1:p)
    YY <- as.data.frame(Y)
    WW <- as.data.frame(W)
    colnames(YY) <- "Y"
    colnames(WW) <- "W"
    data_dml = data.frame(X, YY, WW)

  out <- list(X = X, Y = Y, W = W, tau = tau, m = m, e = e, dgp = dgp, dmldat = data_dml)

  out
}

### 2.1. Simulation - Honest and Dishonest Causal Forest for ATEs

In [ ]:
set.seed(123)     # random seed for replicability of results 
mse.reps <- 2500  # number of repetitions: set to 2500
n <- 200          # number of observations
p <- 30           # number of covariates
tau <- 1          # true treatment effect is constant in all setups

# DGPs with different levels of confounding 
dgp <- c("nw2","aw1","nw1","nw3")
grid <- expand.grid(n = n, p = p, dgp = dgp, stringsAsFactors = FALSE)
var_matrix <- matrix(NA, ncol = length(n)*length(p), nrow = mse.reps)

# Function to apply GRF on DGPs and output estimates of tau
out <- lapply(1:nrow(grid), function(i) {
  n <- grid$n[i]
  p <- grid$p[i]
  dgp <- grid$dgp[i]
  tau.hat <- replicate(mse.reps, {
    data <- generate_DGP(n = n, p = p, dgp = dgp, sigma.tau = 1)
    data.test <- generate_DGP(n = 1000, p = p, dgp = dgp, sigma.tau = 1)
    cf_honest      <- causal_forest(data$X, data$Y, data$W, honesty = TRUE)    # honest forest
    cf_not_honest  <- causal_forest(data$X, data$Y, data$W, honesty = FALSE)
    tau_honest     <- average_treatment_effect(cf_honest, target.sample = "all",  method = "TMLE")[1]
    tau_not_honest <- average_treatment_effect(cf_not_honest, target.sample = "all",  method = "TMLE")[1]
    tau_estims <- cbind(tau_honest,tau_not_honest)

  })

  data.frame(dgp = dgp, p = p, n = n, C.GRF = tau.hat)
})

# Dataframe containing estimates of tau across all simulations
tau_table <- do.call(rbind, out)

In [ ]:
# Separate collective results into separate matrices
mat1 <- as.matrix(tau_table[,4:ncol(tau_table)])
colnames(mat1) <- NULL
rownames(mat1) <- NULL

# Each matrix corresponds to iterations for honest/dishonest forest given DGP 1-4
tau.estims_h1  <- mat1[1,c(seq(1,ncol(mat1),by=2))]
tau.estims_nh1 <- mat1[1,c(seq(2,ncol(mat1),by=2))]

tau.estims_h2  <- mat1[2,c(seq(1,ncol(mat1),by=2))]
tau.estims_nh2 <- mat1[2,c(seq(2,ncol(mat1),by=2))]

tau.estims_h3  <- mat1[3,c(seq(1,ncol(mat1),by=2))]
tau.estims_nh3 <- mat1[3,c(seq(2,ncol(mat1),by=2))]

tau.estims_h4  <- mat1[4,c(seq(1,ncol(mat1),by=2))]
tau.estims_nh4 <- mat1[4,c(seq(2,ncol(mat1),by=2))]

In [ ]:
# Mean of tau estimates across all repetitions 

###### Mean
honest_1  <- mean(tau.estims_h1)
nhonest_1 <- mean(tau.estims_nh1)

honest_2  <- mean(tau.estims_h2)
nhonest_2 <- mean(tau.estims_nh2)

honest_3  <- mean(tau.estims_h3)
nhonest_3 <- mean(tau.estims_nh3)

honest_4  <- mean(tau.estims_h4)
nhonest_4 <- mean(tau.estims_nh4)

######## Bias 
honest_1_bias  <- mean(tau.estims_h1 - tau)
nhonest_1_bias <- mean(tau.estims_nh1 - tau)

honest_2_bias  <- mean(tau.estims_h2 - tau)
nhonest_2_bias <- mean(tau.estims_nh2 - tau)

honest_3_bias <- mean(tau.estims_h3 - tau)
nhonest_3_bias <- mean(tau.estims_nh3 - tau)

honest_4_bias  <- mean(tau.estims_h4 - tau)
nhonest_4_bias <- mean(tau.estims_nh4 - tau)

bias_ate <- rbind(cbind(honest_1_bias,nhonest_1_bias),
                  cbind(honest_2_bias,nhonest_2_bias),
                  cbind(honest_3_bias,nhonest_3_bias),
                  cbind(honest_4_bias,nhonest_4_bias))
                  

######### Variance 
honest_1_var  <- mean((tau.estims_h1 - mean(tau.estims_h1))^2)
nhonest_1_var <- mean((tau.estims_nh1 - mean(tau.estims_nh1))^2)

honest_2_var <- mean((tau.estims_h2 - mean(tau.estims_h2))^2)
nhonest_2_var <- mean((tau.estims_nh2 - mean(tau.estims_nh2))^2)

honest_3_var <- mean((tau.estims_h3 - mean(tau.estims_h3))^2)
nhonest_3_var <- mean((tau.estims_nh3 - mean(tau.estims_nh3))^2)

honest_4_var  <- mean((tau.estims_h4 - mean(tau.estims_h4))^2)
nhonest_4_var <- mean((tau.estims_nh4 - mean(tau.estims_nh4))^2)


var_ate <- rbind(cbind(honest_1_var,nhonest_1_var),
                  cbind(honest_2_var,nhonest_2_var),
                  cbind(honest_3_var,nhonest_3_var),
                  cbind(honest_4_var,nhonest_4_var))


######### MSE 
honest_1_mse  <- mean((tau.estims_h1 - tau)^2) 
nhonest_1_mse <- mean((tau.estims_nh1 - tau)^2) 

honest_2_mse <- mean((tau.estims_h2 - tau)^2) 
nhonest_2_mse <- mean((tau.estims_nh2 - tau)^2) 

honest_3_mse <- mean((tau.estims_h3 - tau)^2) 
nhonest_3_mse <- mean((tau.estims_nh3 - tau)^2) 

honest_4_mse  <- mean((tau.estims_h4 - tau)^2) 
nhonest_4_mse <- mean((tau.estims_nh4 - tau)^2) 

mse_ate <- rbind(cbind(honest_1_mse,nhonest_1_mse),
                  cbind(honest_2_mse,nhonest_2_mse),
                  cbind(honest_3_mse,nhonest_3_mse),
                  cbind(honest_4_mse,nhonest_4_mse))


In [ ]:
data.frame(cbind("Bias" = round(bias_ate*100,2), 
                                "VAR" = round(var_ate*100,2),
                                "MSE" = round(mse_ate*100,2)))

In [ ]:
results_ate <- data.frame(cbind("Bias" = round(bias_ate*100,2), 
                                "VAR" = round(var_ate*100,2),
                                "MSE" = round(mse_ate*100,2)))


In [ ]:
results_ate_latex <- cbind(results_ate[,1],results_ate[,3],results_ate[,5],
                           results_ate[,2],results_ate[,4],results_ate[,6])

#print.xtable(xtable(head(results_ate_latex)), file = "./Data.txt")


In [ ]:
# Plotting the distribution of scaled estimation for Honest and Dishonest Forest across all 5 DGPs.

# Honest Forest Case 1
df_h1 = data.frame('tau.estims_h1' = tau.estims_h1 - 1)
sd_h1 = sd(tau.estims_h1)

df_h1 = data.frame("tau.estims_h1" = df_h1$tau.estims_h1/sd_h1)

figa = ggplot(df_h1, aes(x = tau.estims_h1)) +
        geom_histogram(aes(y=..density..), bins = 100, fill = "royalblue3", alpha = 0.3, color = "dark blue") +
        geom_vline(aes(xintercept = 0), col = "black") +
        xlim(c(-10, 10)) + xlab("") + ylab("") + theme_minimal() + 
        theme(plot.title = element_text(color="black", face="bold", size=30),axis.text=element_text(size=20)) +
        stat_function(fun = dnorm, args = list(mean = 0, sd = 1), geom = "area", col = "red", fill = "red", alpha = 0.01) + 
        ggtitle(paste0("Scaled Estimation Error n = ", 200, ", p = ", 30)) + theme(plot.title = element_text(size=30,face="bold", hjust = 0.5))
options(repr.plot.width=30, repr.plot.height=14)

figa

# Dishonest Forest Case 1
df_nh1 = data.frame('tau.estims_nh1' = tau.estims_nh1 - 1)
sd_nh1 = sd(tau.estims_nh1)

df_nh1 = data.frame("tau.estims_nh1" = df_nh1$tau.estims_nh1/sd_nh1)

figb = ggplot(df_nh1, aes(x = tau.estims_nh1)) +
        geom_histogram(aes(y=..density..), bins = 100, fill = "royalblue3", alpha = 0.3, color = "dark blue") +
        geom_vline(aes(xintercept = 0), col = "black") +
        xlim(c(-10, 10)) + xlab("") + ylab("") + theme_minimal() + 
        theme(plot.title = element_text(color="black", face="bold", size=30),axis.text=element_text(size=20)) +
        stat_function(fun = dnorm, args = list(mean = 0, sd = 1), geom = "area", col = "red", fill = "red", alpha = 0.01) + 
        ggtitle(paste0("Scaled Estimation Error n = ", 200, ", p = ", 30)) + theme(plot.title = element_text(size=30,face="bold", hjust = 0.5))
options(repr.plot.width=30, repr.plot.height=14)

figb

# Honest Forest Case 2
df_h2 = data.frame('tau.estims_h2' = tau.estims_h2 - 1)
sd_h2 = sd(tau.estims_h2)

df_h2 = data.frame("tau.estims_h2" = df_h2$tau.estims_h2/sd_h2)

figc = ggplot(df_h2, aes(x = tau.estims_h2)) +
        geom_histogram(aes(y=..density..), bins = 100, fill = "royalblue3", alpha = 0.3, color = "dark blue") +
        geom_vline(aes(xintercept = 0), col = "black") +
        xlim(c(-10, 10)) + xlab("") + ylab("") + theme_minimal() + 
        theme(plot.title = element_text(color="black", face="bold", size=30),axis.text=element_text(size=20)) +
        stat_function(fun = dnorm, args = list(mean = 0, sd = 1), geom = "area", col = "red", fill = "red", alpha = 0.01)  
        #ggtitle(paste0("Scaled Estimation Error n = ", 200, ", p = ", 30)) + theme(plot.title = element_text(face="bold", hjust = 0.5))
options(repr.plot.width=30, repr.plot.height=14)

figc

# Dishonest Forest Case 2
df_nh2 = data.frame('tau.estims_nh2' = tau.estims_nh2 - 1)
sd_nh2 = sd(tau.estims_nh2)

df_nh2 = data.frame("tau.estims_nh2" = df_nh2$tau.estims_nh2/sd_nh2)

figd = ggplot(df_nh2, aes(x = tau.estims_nh2)) +
        geom_histogram(aes(y=..density..), bins = 100, fill = "royalblue3", alpha = 0.3, color = "dark blue") +
        geom_vline(aes(xintercept = 0), col = "black") +
        xlim(c(-10, 10)) + xlab("") + ylab("") + theme_minimal() + 
        theme(plot.title = element_text(color="black", face="bold", size=30),axis.text=element_text(size=20)) +
        stat_function(fun = dnorm, args = list(mean = 0, sd = 1), geom = "area", col = "red", fill = "red", alpha = 0.01)  
       # ggtitle(paste0("Scaled Estimation Error n = ", 200, ", p = ", 30)) + theme(plot.title = element_text(face="bold", hjust = 0.5))
options(repr.plot.width=30, repr.plot.height=14)

figd

# Honest Forest Case 3
df_h3 = data.frame('tau.estims_h3' = tau.estims_h3 - 1)
sd_h3 = sd(tau.estims_h3)

df_h3 = data.frame("tau.estims_h3" = df_h3$tau.estims_h3/sd_h3)

fige = ggplot(df_h3, aes(x = tau.estims_h3)) +
        geom_histogram(aes(y=..density..), bins = 100, fill = "royalblue3", alpha = 0.3, color = "dark blue") +
        geom_vline(aes(xintercept = 0), col = "black") +
        xlim(c(-10, 10)) + xlab("") + ylab("") + theme_minimal() + 
        theme(plot.title = element_text(color="black", face="bold", size=30),axis.text=element_text(size=20)) +
        stat_function(fun = dnorm, args = list(mean = 0, sd = 1), geom = "area", col = "red", fill = "red", alpha = 0.01)  
        #ggtitle(paste0("Scaled Estimation Error n = ", 200, ", p = ", 30)) + theme(plot.title = element_text(face="bold", hjust = 0.5))
options(repr.plot.width=30, repr.plot.height=14)

fige

# Dishonest Forest Case 3
df_nh3 = data.frame('tau.estims_nh3' = tau.estims_nh3 - 1)
sd_nh3 = sd(tau.estims_nh3)

df_nh3 = data.frame("tau.estims_nh3" = df_nh3$tau.estims_nh3/sd_nh3)

figf = ggplot(df_nh3, aes(x = tau.estims_nh3)) +
        geom_histogram(aes(y=..density..), bins = 100, fill = "royalblue3", alpha = 0.3, color = "dark blue") +
        geom_vline(aes(xintercept = 0), col = "black") +
        xlim(c(-10, 10)) + xlab("") + ylab("") + theme_minimal() + 
        theme(plot.title = element_text(color="black", face="bold", size=30),axis.text=element_text(size=20)) +
        stat_function(fun = dnorm, args = list(mean = 0, sd = 1), geom = "area", col = "red", fill = "red", alpha = 0.01) 
        #ggtitle(paste0("Scaled Estimation Error n = ", 200, ", p = ", 30)) + theme(plot.title = element_text(face="bold", hjust = 0.5))
options(repr.plot.width=30, repr.plot.height=14)

figf

# Honest Forest Case 4
df_h4 = data.frame('tau.estims_h4' = tau.estims_h4 - 1)
sd_h4 = sd(tau.estims_h4)

df_h4 = data.frame("tau.estims_h4" = df_h4$tau.estims_h4/sd_h4)

figg = ggplot(df_h4, aes(x = tau.estims_h4)) +
        geom_histogram(aes(y=..density..), bins = 100, fill = "royalblue3", alpha = 0.3, color = "dark blue") +
        geom_vline(aes(xintercept = 0), col = "black") +
        xlim(c(-10, 10)) + xlab("") + ylab("") + theme_minimal() + 
        theme(plot.title = element_text(color="black", face="bold", size=30),axis.text=element_text(size=20)) +
        stat_function(fun = dnorm, args = list(mean = 0, sd = 1), geom = "area", col = "red", fill = "red", alpha = 0.01) 
       # ggtitle(paste0("Scaled Estimation Error n = ", 200, ", p = ", 30)) + theme(plot.title = element_text(face="bold", hjust = 0.5))
options(repr.plot.width=30, repr.plot.height=14)

figg

# Dishonest Forest Case 4
df_nh4 = data.frame('tau.estims_nh4' = tau.estims_nh4 - 1)
sd_nh4 = sd(tau.estims_nh4)

df_nh4 = data.frame("tau.estims_nh4" = df_nh4$tau.estims_nh4/sd_nh4)

figh = ggplot(df_nh4, aes(x = tau.estims_nh4)) +
        geom_histogram(aes(y=..density..), bins = 100, fill = "royalblue3", alpha = 0.3, color = "dark blue") +
        geom_vline(aes(xintercept = 0), col = "black") +
        xlim(c(-10, 10)) + xlab("") + ylab("") + theme_minimal() + 
        theme(plot.title = element_text(color="black", face="bold", size=30),axis.text=element_text(size=20)) +
        stat_function(fun = dnorm, args = list(mean = 0, sd = 1), geom = "area", col = "red", fill = "red", alpha = 0.01)  
       # ggtitle(paste0("Scaled Estimation Error n = ", 200, ", p = ", 30)) + theme(plot.title = element_text(face="bold", hjust = 0.5))
options(repr.plot.width=30, repr.plot.height=14)

figh



### 2.3. Causal Forests, Double Machine Learning, and Post Double Selection

In [ ]:
# LASSO Estimation Function
est_lasso <- function(obj_dml_data) {
    
   # set.seed(70)
    learner <- lrn("regr.cv_glmnet", s="lambda.min")
    g_hat <- learner$clone()                                  # use LASSO to estimate nuisance g() and m()
    m_hat <- learner$clone()
    
    DML_PLR  <- DoubleMLPLR$new(obj_dml_data, g_hat, m_hat)   # initialize the DML Partially Linear Model 
    DML_PLR$fit()                                             # Fit the model and extract coefficient estimates
    theta_hat <- DML_PLR$coef
    sd_hat    <- DML_PLR$se
    params    <- cbind(theta_hat,sd_hat)
    return(params)
}

est_rforest <- function(obj_dml_data) {
    
   # set.seed(70)
    learner <- lrn("regr.ranger", num.trees=100, min.node.size=2, max.depth=5)
    g_hat <- learner$clone()                                  # use LASSO to estimate nuisance g() and m()
    m_hat <- learner$clone()
    
    DML_PLR  <- DoubleMLPLR$new(obj_dml_data, g_hat, m_hat)   # initialize the DML Partially Linear Model 
    DML_PLR$fit()                                             # Fit the model and extract coefficient estimates
    theta_hat <- DML_PLR$coef
    sd_hat    <- DML_PLR$se
    params    <- cbind(theta_hat,sd_hat)
    return(params)
}


sim_function1 <- function(data){
    
    DML_data     <- double_ml_data_from_data_frame(data, y_col = "Y", d_cols = "W")
    param_output <- est_lasso(DML_data) 
    return(param_output)
}

sim_function2 <- function(data){
    
    DML_data     <- double_ml_data_from_data_frame(data, y_col = "Y", d_cols = "W")
    param_output <- est_rforest(DML_data) 
    return(param_output)
}


In [ ]:
set.seed(123)     # random seed for replicability of results 
mse.reps <- 1000    # number of repetitions: set to 1000
n <- 200          # number of observations
p <- c(30, 50, 100, 200)           # number of covariates
tau <- 1          # true treatment effect is constant in all setups

# DGPs with different levels of confounding 
dgp <- c("nw2","aw1","nw1","nw3")
grid <- expand.grid(n = n, p = p, dgp = dgp, stringsAsFactors = FALSE)
var_matrix <- matrix(NA, ncol = length(n)*length(p), nrow = mse.reps)

# Function to apply GRF on DGPs and output estimates of tau
out <- lapply(1:nrow(grid), function(i) {
  n <- grid$n[i]
  p <- grid$p[i]
  dgp <- grid$dgp[i]
  tau.hat <- replicate(mse.reps, {
    data <- generate_DGP(n = n, p = p, dgp = dgp, sigma.tau = 1)
    theta_estim1 <- sim_function1(data$dmldat)[1]  
    theta_estim2 <- sim_function2(data$dmldat)[1] 
    cf_honest   <- causal_forest(data$X, data$Y, data$W, honesty = TRUE)    # honest forest
    tau_honest  <- suppressWarnings(average_treatment_effect(cf_honest))[1]
    DS          <- summary(rlassoEffect(x = data$X, y = data$Y, d=data$W, method = "double selection"))$coefficients[1,1]

    tau_estims <- cbind(tau_honest,theta_estim1,theta_estim2,DS)

  })

  data.frame(dgp = dgp, p = p, n = n, C.GRF = tau.hat)
})

# Dataframe containing estimates of tau across all simulations
tau_table_comp <- do.call(rbind, out)

In [ ]:
# Separate large data into separate dataframes
tau_all <- tau_table_comp
tau_all <- tau_all[order(tau_all$p),]

dim_30  <- tau_all[1:4,]
dim_50  <- tau_all[5:8,]
dim_100 <- tau_all[9:12,]
dim_200 <- tau_all[13:16,]

In [ ]:
# Separate collective results into separate matrices
mat_comp_30 <- as.matrix(dim_30[,4:ncol(dim_30)])
colnames(mat_comp_30) <- NULL
rownames(mat_comp_30) <- NULL

# Each matrix corresponds to iterations for honest/dishonest forest given DGP 1-6
estims_grf_30 <- cbind(mat_comp_30[1,c(seq(1,ncol(mat_comp_30),by=4))],
                       mat_comp_30[2,c(seq(1,ncol(mat_comp_30),by=4))],
                       mat_comp_30[3,c(seq(1,ncol(mat_comp_30),by=4))],
                       mat_comp_30[4,c(seq(1,ncol(mat_comp_30),by=4))])

estims_dmll_30 <- cbind(mat_comp_30[1,c(seq(2,ncol(mat_comp_30),by=4))],
                        mat_comp_30[2,c(seq(2,ncol(mat_comp_30),by=4))],
                        mat_comp_30[3,c(seq(2,ncol(mat_comp_30),by=4))],
                        mat_comp_30[4,c(seq(2,ncol(mat_comp_30),by=4))])

estims_dmlr_30 <- cbind(mat_comp_30[1,c(seq(3,ncol(mat_comp_30),by=4))],
                        mat_comp_30[2,c(seq(3,ncol(mat_comp_30),by=4))],
                        mat_comp_30[3,c(seq(3,ncol(mat_comp_30),by=4))],
                        mat_comp_30[4,c(seq(3,ncol(mat_comp_30),by=4))])

estims_pds_30 <- cbind(mat_comp_30[1,c(seq(4,ncol(mat_comp_30),by=4))],
                       mat_comp_30[2,c(seq(4,ncol(mat_comp_30),by=4))],
                       mat_comp_30[3,c(seq(4,ncol(mat_comp_30),by=4))],
                       mat_comp_30[4,c(seq(4,ncol(mat_comp_30),by=4))])

# Separate collective results into separate matrices
mat_comp_50 <- as.matrix(dim_50[,4:ncol(dim_50)])
colnames(mat_comp_50) <- NULL
rownames(mat_comp_50) <- NULL

# Each matrix corresponds to iterations for honest/dishonest forest given DGP 1-4
estims_grf_50 <- cbind(mat_comp_50[1,c(seq(1,ncol(mat_comp_50),by=4))],
                       mat_comp_50[2,c(seq(1,ncol(mat_comp_50),by=4))],
                       mat_comp_50[3,c(seq(1,ncol(mat_comp_50),by=4))],
                       mat_comp_50[4,c(seq(1,ncol(mat_comp_50),by=4))])

estims_dmll_50 <- cbind(mat_comp_50[1,c(seq(2,ncol(mat_comp_50),by=4))],
                        mat_comp_50[2,c(seq(2,ncol(mat_comp_50),by=4))],
                        mat_comp_50[3,c(seq(2,ncol(mat_comp_50),by=4))],
                        mat_comp_50[4,c(seq(2,ncol(mat_comp_50),by=4))])

estims_dmlr_50 <- cbind(mat_comp_50[1,c(seq(3,ncol(mat_comp_50),by=4))],
                        mat_comp_50[2,c(seq(3,ncol(mat_comp_50),by=4))],
                        mat_comp_50[3,c(seq(3,ncol(mat_comp_50),by=4))],
                        mat_comp_50[4,c(seq(3,ncol(mat_comp_50),by=4))])

estims_pds_50 <- cbind(mat_comp_50[1,c(seq(4,ncol(mat_comp_50),by=4))],
                       mat_comp_50[2,c(seq(4,ncol(mat_comp_50),by=4))],
                       mat_comp_50[3,c(seq(4,ncol(mat_comp_50),by=4))],
                       mat_comp_50[4,c(seq(4,ncol(mat_comp_50),by=4))])

# Separate collective results into separate matrices
mat_comp_100 <- as.matrix(dim_100[,4:ncol(dim_100)])
colnames(mat_comp_100) <- NULL
rownames(mat_comp_100) <- NULL

# Each matrix corresponds to iterations for honest/dishonest forest given DGP 1-4
estims_grf_100 <- cbind(mat_comp_100[1,c(seq(1,ncol(mat_comp_100),by=4))],
                        mat_comp_100[2,c(seq(1,ncol(mat_comp_100),by=4))],
                        mat_comp_100[3,c(seq(1,ncol(mat_comp_100),by=4))],
                        mat_comp_100[4,c(seq(1,ncol(mat_comp_100),by=4))])

estims_dmll_100 <- cbind(mat_comp_100[1,c(seq(2,ncol(mat_comp_100),by=4))],
                         mat_comp_100[2,c(seq(2,ncol(mat_comp_100),by=4))],
                         mat_comp_100[3,c(seq(2,ncol(mat_comp_100),by=4))],
                         mat_comp_100[4,c(seq(2,ncol(mat_comp_100),by=4))])

estims_dmlr_100 <- cbind(mat_comp_100[1,c(seq(3,ncol(mat_comp_100),by=4))],
                         mat_comp_100[2,c(seq(3,ncol(mat_comp_100),by=4))],
                         mat_comp_100[3,c(seq(3,ncol(mat_comp_100),by=4))],
                         mat_comp_100[4,c(seq(3,ncol(mat_comp_100),by=4))])

estims_pds_100 <- cbind(mat_comp_100[1,c(seq(4,ncol(mat_comp_100),by=4))],
                        mat_comp_100[2,c(seq(4,ncol(mat_comp_100),by=4))],
                        mat_comp_100[3,c(seq(4,ncol(mat_comp_100),by=4))],
                        mat_comp_100[4,c(seq(4,ncol(mat_comp_100),by=4))])

# Separate collective results into separate matrices
mat_comp_200 <- as.matrix(dim_200[,4:ncol(dim_200)])
colnames(mat_comp_200) <- NULL
rownames(mat_comp_200) <- NULL

# Each matrix corresponds to iterations for honest/dishonest forest given DGP 1-4
estims_grf_200 <- cbind(mat_comp_200[1,c(seq(1,ncol(mat_comp_200),by=4))],
                        mat_comp_200[2,c(seq(1,ncol(mat_comp_200),by=4))],
                        mat_comp_200[3,c(seq(1,ncol(mat_comp_200),by=4))],
                        mat_comp_200[4,c(seq(1,ncol(mat_comp_200),by=4))])

estims_dmll_200 <- cbind(mat_comp_200[1,c(seq(2,ncol(mat_comp_200),by=4))],
                         mat_comp_200[2,c(seq(2,ncol(mat_comp_200),by=4))],
                         mat_comp_200[3,c(seq(2,ncol(mat_comp_200),by=4))],
                         mat_comp_200[4,c(seq(2,ncol(mat_comp_200),by=4))])

estims_dmlr_200 <- cbind(mat_comp_200[1,c(seq(3,ncol(mat_comp_200),by=4))],
                         mat_comp_200[2,c(seq(3,ncol(mat_comp_200),by=4))],
                         mat_comp_200[3,c(seq(3,ncol(mat_comp_200),by=4))],
                         mat_comp_200[4,c(seq(3,ncol(mat_comp_200),by=4))])

estims_pds_200 <- cbind(mat_comp_200[1,c(seq(4,ncol(mat_comp_200),by=4))],
                        mat_comp_200[2,c(seq(4,ncol(mat_comp_200),by=4))],
                        mat_comp_200[3,c(seq(4,ncol(mat_comp_200),by=4))],
                        mat_comp_200[4,c(seq(4,ncol(mat_comp_200),by=4))])

In [ ]:
# Mean of tau estimates across all repetitions 
bias_table_30    <- matrix(NA, nrow = 4, ncol = 4)
var_table_30     <- matrix(NA, nrow = 4, ncol = 4)
mse_table_30     <- matrix(NA, nrow = 4, ncol = 4)

bias_table_50    <- matrix(NA, nrow = 4, ncol = 4)
var_table_50     <- matrix(NA, nrow = 4, ncol = 4)
mse_table_50     <- matrix(NA, nrow = 4, ncol = 4)

bias_table_100    <- matrix(NA, nrow = 4, ncol = 4)
var_table_100     <- matrix(NA, nrow = 4, ncol = 4)
mse_table_100     <- matrix(NA, nrow = 4, ncol = 4)

bias_table_200    <- matrix(NA, nrow = 4, ncol = 4)
var_table_200     <- matrix(NA, nrow = 4, ncol = 4)
mse_table_200     <- matrix(NA, nrow = 4, ncol = 4)


for (i in 1:4) {
    
    bias_table_30[i,1] <- mean(estims_grf_30[,i] - tau)
    bias_table_30[i,2] <- mean(estims_dmll_30[,i] - tau)
    bias_table_30[i,3] <- mean(estims_dmlr_30[,i] - tau)
    bias_table_30[i,4] <- mean(estims_pds_30[,i] - tau)
    
    bias_table_50[i,1] <- mean(estims_grf_50[,i] - tau)
    bias_table_50[i,2] <- mean(estims_dmll_50[,i] - tau)
    bias_table_50[i,3] <- mean(estims_dmlr_50[,i] - tau)
    bias_table_50[i,4] <- mean(estims_pds_50[,i] - tau)
    
    bias_table_100[i,1] <- mean(estims_grf_100[,i] - tau)
    bias_table_100[i,2] <- mean(estims_dmll_100[,i] - tau)
    bias_table_100[i,3] <- mean(estims_dmlr_100[,i] - tau)
    bias_table_100[i,4] <- mean(estims_pds_100[,i] - tau)
    
    bias_table_200[i,1] <- mean(estims_grf_200[,i] - tau)
    bias_table_200[i,2] <- mean(estims_dmll_200[,i] - tau)
    bias_table_200[i,3] <- mean(estims_dmlr_200[,i] - tau)
    bias_table_200[i,4] <- mean(estims_pds_200[,i] - tau)
    
    var_table_30[i,1] <- mean((estims_grf_30[,i] - mean(estims_grf_30[,i]))^2)
    var_table_30[i,2] <- mean((estims_dmll_30[,i] - mean(estims_dmll_30[,i]))^2)
    var_table_30[i,3] <- mean((estims_dmlr_30[,i] - mean(estims_dmlr_30[,i]))^2)
    var_table_30[i,4] <- mean((estims_pds_30[,i] - mean(estims_pds_30[,i]))^2)
    
    var_table_50[i,1] <- mean((estims_grf_50[,i] - mean(estims_grf_50[,i]))^2)
    var_table_50[i,2] <- mean((estims_dmll_50[,i] - mean(estims_dmll_50[,i]))^2)
    var_table_50[i,3] <- mean((estims_dmlr_50[,i] - mean(estims_dmlr_50[,i]))^2)
    var_table_50[i,4] <- mean((estims_pds_50[,i] - mean(estims_pds_50[,i]))^2)
          
    var_table_100[i,1] <- mean((estims_grf_100[,i] - mean(estims_grf_100[,i]))^2)
    var_table_100[i,2] <- mean((estims_dmll_100[,i] - mean(estims_dmll_100[,i]))^2)
    var_table_100[i,3] <- mean((estims_dmlr_100[,i] - mean(estims_dmlr_100[,i]))^2)
    var_table_100[i,4] <- mean((estims_pds_100[,i] - mean(estims_pds_100[,i]))^2)
        
    var_table_200[i,1] <- mean((estims_grf_200[,i] - mean(estims_grf_200[,i]))^2)
    var_table_200[i,2] <- mean((estims_dmll_200[,i] - mean(estims_dmll_200[,i]))^2)
    var_table_200[i,3] <- mean((estims_dmlr_200[,i] - mean(estims_dmlr_200[,i]))^2)
    var_table_200[i,4] <- mean((estims_pds_200[,i] - mean(estims_pds_200[,i]))^2)
    
    mse_table_30[i,1] <- mean((estims_grf_30[,i] - tau)^2) 
    mse_table_30[i,2] <- mean((estims_dmll_30[,i] - tau)^2) 
    mse_table_30[i,3] <- mean((estims_dmlr_30[,i] - tau)^2)
    mse_table_30[i,4] <- mean((estims_pds_30[,i] - tau)^2) 
    
    mse_table_50[i,1] <- mean((estims_grf_50[,i] - tau)^2) 
    mse_table_50[i,2] <- mean((estims_dmll_50[,i] - tau)^2) 
    mse_table_50[i,3] <- mean((estims_dmlr_50[,i] - tau)^2)
    mse_table_50[i,4] <- mean((estims_pds_50[,i] - tau)^2)
                              
    mse_table_100[i,1] <- mean((estims_grf_100[,i] - tau)^2) 
    mse_table_100[i,2] <- mean((estims_dmll_100[,i] - tau)^2) 
    mse_table_100[i,3] <- mean((estims_dmlr_100[,i] - tau)^2)
    mse_table_100[i,4] <- mean((estims_pds_100[,i] - tau)^2)
                              
    mse_table_200[i,1] <- mean((estims_grf_200[,i] - tau)^2) 
    mse_table_200[i,2] <- mean((estims_dmll_200[,i] - tau)^2) 
    mse_table_200[i,3] <- mean((estims_dmlr_200[,i] - tau)^2)
    mse_table_200[i,4] <- mean((estims_pds_200[,i] - tau)^2)
}




In [ ]:
bias_table_30 <- round(bias_table_30*10,3) 
var_table_30 <- round(var_table_30*10,3) 
mse_table_30 <- round(mse_table_30* 10,3) 

bias_table_50 <- round(bias_table_50*10,3) 
var_table_50 <- round(var_table_50*10,3) 
mse_table_50 <- round(mse_table_50* 10,3) 

bias_table_100 <- round(bias_table_100*10,3) 
var_table_100 <- round(var_table_100*10,3) 
mse_table_100 <- round(mse_table_100* 10,3) 

bias_table_200 <- round(bias_table_200*10,3) 
var_table_200  <- round(var_table_200*10 ,3) 
mse_table_200  <- round(mse_table_200 * 10,3) 

In [ ]:
results_gdp_30 <- cbind(bias_table_30[,1],var_table_30[,1],mse_table_30[,1],
                        bias_table_30[,2],var_table_30[,2],mse_table_30[,2],
                        bias_table_30[,3],var_table_30[,3],mse_table_30[,3],
                        bias_table_30[,4],var_table_30[,4],mse_table_30[,4])

results_gdp_50 <- cbind(bias_table_50[,1],var_table_50[,1],mse_table_50[,1],
                        bias_table_50[,2],var_table_50[,2],mse_table_50[,2],
                        bias_table_50[,3],var_table_50[,3],mse_table_50[,3],
                        bias_table_50[,4],var_table_50[,4],mse_table_50[,4])

results_gdp_100 <- cbind(bias_table_100[,1],var_table_100[,1],mse_table_100[,1],
                         bias_table_100[,2],var_table_100[,2],mse_table_100[,2],
                         bias_table_100[,3],var_table_100[,3],mse_table_100[,3],
                         bias_table_100[,4],var_table_100[,4],mse_table_100[,4])

results_gdp_200 <- cbind(bias_table_200[,1],var_table_200[,1],mse_table_200[,1],
                         bias_table_200[,2],var_table_200[,2],mse_table_200[,2],
                         bias_table_200[,3],var_table_200[,3],mse_table_200[,3],
                         bias_table_200[,4],var_table_200[,4],mse_table_200[,4])
 

In [ ]:
# This saves the results locally
#print.xtable(xtable(head(results_gdp_30), digits = 2), file = "./results_gdp_30")
#print.xtable(xtable(head(results_gdp_50), digits = 2), file = "./results_gdp_50")
#print.xtable(xtable(head(results_gdp_100), digits = 2), file = "./results_gdp_100")
#print.xtable(xtable(head(results_gdp_200), digits = 2), file = "./results_gdp_200")

### 3. Predicting HTEs 

### 3.1. Data Generating Process 3

In [ ]:
generate_data_het <- function(n, p, sigma.m = 1, sigma.tau = 0.1, sigma.noise = 1,
                                 dgp = c("aw1", "aw3", "ai2",
                                         "nw1", "nw2", "nw3")) {
  # To add an additonal DGP, fill in the template below and add an entry to `dgp` and `.minp`.
  .minp <- c(aw1=2, aw3=1, ai2=6, nw1=5, nw2=5, nw3=3, nw4=5)
  dgp <- match.arg(dgp)
  minp <- .minp[dgp]
  if (p < minp) {
    msg <- paste0("Selected dgp ", dgp, " requires a minimum of ", minp, " variables.")
    stop(msg)
  }

  # Create data
    if (dgp == "aw1") {
    # equation (27) of https://arxiv.org/pdf/1510.04342.pdf
    X <- matrix(runif(n * p, min = 0, max = 1), n, p)
    tau <- rep(0, n)  # Treatment effect is zero
    e <- (1 / 4)  * (exp(X[,3]) /(1 + exp(X[,3])))
    W <- rbinom(n = n, size = 1, prob = e)
    m <- 2 * X[, 1] - 1 + e * tau
    V <- 1
  } else if (dgp == "aw3") {
    # section 6.2 in https://arxiv.org/pdf/1610.01271.pdf
    # (confounding from aw1, tau from aw2)
    X <- matrix(runif(n * p), n, p)
    zeta1 <- 1 + 1 / (1 + exp(-20 * (X[, 1] - (1 / 3))))
    zeta2 <- 1 + 1 / (1 + exp(-20 * (X[, 2] - (1 / 3))))
    tau <- zeta1 * zeta2
    e <- (1 / 4)  * (exp(X[,3]) /(1 + exp(X[,3])))
    W <- rbinom(n = n, size = 1, prob = e)
    m <- 2 * X[, 1] - 1 + e * tau
    V <- 1
  } else if (dgp == "ai2") {
    X <- matrix(runif(n * p), n, p)
    nu_x <- 0.5 * X[, 1] + 0.5 * X[, 2] + X[, 3] + X[, 4] + X[, 5] + X[, 6]
    tau <- 0.5 * ((X[, 1] > 0) * X[, 1] + (X[, 2] > 0) * X[, 2])
    e <- rep(0.5, n)
    W <- rbinom(n = n, size = 1, prob = e)
    m <- nu_x + e * tau
    V <- 0.1^2
  } else if (dgp == "nw1") {
    # "Setup A" from Section 4 of https://arxiv.org/pdf/1712.04912.pdf
    # Difficult nuisance components, easy treatment effect function.
    X <- matrix(runif(n * p), n, p)
    tau <- (X[, 1] + X[, 2]) / 2
    eta <- 0.1
    e <- pmax(eta, pmin(sin(pi * X[, 1] * X[, 2]), 1 - eta))
    W <- rbinom(n = n, size = 1, prob = e)
    m <- sin(pi * X[, 1] * X[, 2]) + 2 * (X[, 3] - 0.5)^2 + X[, 4] + 0.5 * X[, 5] + e * tau
    V <- 1
  } else if (dgp == "nw2") {
    # "Setup B" from Section 4 of https://arxiv.org/pdf/1712.04912.pdf
    # Randomized trial
    X <- matrix(runif(n * p), n, p)
    tau <- X[,1] + log(1 + exp(X[, 2]))
    e <- rep(0.5, n)
    W <- rbinom(n = n, size = 1, prob = e)
    m <- pmax(0, X[, 1] + X[, 2], X[, 3]) + pmax(0, X[, 4] + X[, 5]) + e * tau
    V <- 1
  } else if (dgp == "nw3") {
    # "Setup C" from Section 4 of https://arxiv.org/pdf/1712.04912.pdf
    # Easy propensity score, strong confounding, difficult baseline,
    # constant treatment effect
    X <- matrix(runif(n * p), n, p)
    tau <- X[,1] + log(1 + exp(X[, 2]))
    e <- 1 / (1 + exp(X[, 2] + X[, 3]))
    W <- rbinom(n = n, size = 1, prob = e)
    m <- 2 * log(1 + exp(X[, 1] + X[, 2] + X[, 3])) + e * tau
    V <- 1
  } 

  # Scale and return data (rescale if `m` and `tau` is not constant, the NA check is for when n=1)
  if (!is.na(sd(m)) & !(sd(m) == 0)) {
    m <- m / sd(m) * sigma.m
  }
  if (!is.na(sd(tau)) & !(sd(tau) == 0)) {
    tau <- tau / sd(tau) * sigma.tau
  }
  V <- V / mean(V) * sigma.noise^2
  Y <- m + (W - e) * tau + sqrt(V) * rnorm(n)
  out <- list(X = X, Y = Y, W = W, tau = tau, m = m, e = e, dgp = dgp)

  out
}


### 3.2. Simulation Experiments: Honest and Dishonest Forests for HTEs

### Honest Forest

In [ ]:
set.seed(123)
mse.reps <- 100    # Number of repetitions: Set to 100
n <- c(100, 200, 300)
p <- c(10, 50, 100, 300)

dgp <- c("nw2","ai2","aw1","aw3","nw1","nw3")
#dgp <- c("nw2","aw1","nw1","nw3")
grid <- expand.grid(n = n, p = p, dgp = dgp, stringsAsFactors = FALSE)
var_matrix <- matrix(NA, ncol = length(n)*length(p), nrow = mse.reps)

out <- lapply(1:nrow(grid), function(i) {
  n <- grid$n[i]
  p <- grid$p[i]
  dgp <- grid$dgp[i]
  mse <- replicate(mse.reps, {
    data      <- generate_data_het(n = n, p = p, dgp = dgp, sigma.tau = 1)
    data.test <- generate_data_het(n = 1000, p = p, dgp = dgp, sigma.tau = 1)
    cf        <- causal_forest(data$X, data$Y, data$W)
    tau.hat   <- predict(cf, data.test$X)$predictions
      
    mean((tau.hat - data.test$tau)^2)
  })

  data.frame(dgp = dgp, p = p, n = n, C.GRF = mse * 10)
})

rmse_table <- do.call(rbind, out) # master results stored in rmse_table

### Dishonest Forest

In [ ]:
set.seed(123)
mse.reps <- 100   # Number of repetitions: Set to 100
n <- c(100, 200, 300)
p <- c(10, 50, 100, 300)

dgp <- c("nw2","ai2","aw1","aw3","nw1","nw3")
#dgp <- c("nw2","aw1","nw1","nw3")
grid <- expand.grid(n = n, p = p, dgp = dgp, stringsAsFactors = FALSE)
var_matrix <- matrix(NA, ncol = length(n)*length(p), nrow = mse.reps)

out <- lapply(1:nrow(grid), function(i) {
  n <- grid$n[i]
  p <- grid$p[i]
  dgp <- grid$dgp[i]
  mse <- replicate(mse.reps, {
    data <- generate_data_het(n = n, p = p, dgp = dgp, sigma.tau = 1)
    data.test <- generate_data_het(n = 1000, p = p, dgp = dgp, sigma.tau = 1)
    cf <- causal_forest(data$X, data$Y, data$W, honesty = FALSE)
    tau.hat <- predict(cf, data.test$X)$predictions
      
    mean((tau.hat - data.test$tau)^2)
  })

  data.frame(dgp = dgp, p = p, n = n, C.GRF = mse * 10)
})

rmse_table_2 <- do.call(rbind, out)# master results stored in rmse_table2

#### Output Preparation and Diplay Box-plot replication

In [ ]:
# Melt large data into separate dataframes
box_dat <- matrix(rmse_table[,4],nrow=mse.reps,ncol=72)

MSE1 <-  reshape2::melt(box_dat[,1:12])[,3]
MSE2 <-  reshape2::melt(box_dat[,13:24])[,3]
MSE3 <-  reshape2::melt(box_dat[,25:36])[,3]
MSE4 <-  reshape2::melt(box_dat[,37:48])[,3]
MSE5 <-  reshape2::melt(box_dat[,49:60])[,3]
MSE6 <-  reshape2::melt(box_dat[,61:72])[,3]

SampleSize=rep(rep(n, each=100),4)
dimension=rep(c("P = 10", "P = 50", "P=100", "P=300"),each=300)
data1=data.frame(SampleSize, dimension,MSE1)
data2=data.frame(SampleSize, dimension,MSE2)
data3=data.frame(SampleSize, dimension,MSE3)
data4=data.frame(SampleSize, dimension,MSE4)
data5=data.frame(SampleSize, dimension,MSE5)
data6=data.frame(SampleSize, dimension,MSE6)

In [ ]:
# Separate into individual vectors for calculation of MSE

seq_1 <- seq(1, 1200, by = 100)
mse_table_1 <- rep(NA,12)
mse_table_2 <- rep(NA,12)
mse_table_3 <- rep(NA,12)
mse_table_4 <- rep(NA,12)
mse_table_5 <- rep(NA,12)
mse_table_6 <- rep(NA,12)

for (i in 1:length(seq_1)) {
    mse_table_1[i] <- mean(data1[seq_1[i]:(seq_1[i]+99),3])
    mse_table_2[i] <- mean(data2[seq_1[i]:(seq_1[i]+99),3])
    mse_table_3[i] <- mean(data3[seq_1[i]:(seq_1[i]+99),3])
    mse_table_4[i] <- mean(data4[seq_1[i]:(seq_1[i]+99),3])
    mse_table_5[i] <- mean(data5[seq_1[i]:(seq_1[i]+99),3])
    mse_table_6[i] <- mean(data6[seq_1[i]:(seq_1[i]+99),3])
    
}

In [ ]:
# Combine MSE results into one table 
hte_results <- round(rbind(mse_table_1,mse_table_2,mse_table_3,mse_table_4,mse_table_5,mse_table_6),2)

# and then print table in LateX for export purposes
#print.xtable(xtable(head(hte_results)), file = "./Data1.txt")

#### Redo Previous Steps for Dishonest Forest

In [ ]:
# Melt large data into separate dataframes
box_dat2 <- matrix(rmse_table_2[,4],nrow=mse.reps,ncol=72)

MSE1_2 <- reshape2::melt(box_dat2[,1:12])[,3]
MSE2_2 <- reshape2::melt(box_dat2[,13:24])[,3]
MSE3_2 <- reshape2::melt(box_dat2[,25:36])[,3]
MSE4_2 <- reshape2::melt(box_dat2[,37:48])[,3]
MSE5_2 <- reshape2::melt(box_dat2[,49:60])[,3]
MSE6_2 <- reshape2::melt(box_dat2[,61:72])[,3]

SampleSize=rep(rep(n, each=100),4)
dimension=rep(c("P = 10", "P = 50", "P=100", "P=300"),each=300)

data1_2 = data.frame(SampleSize, dimension,MSE1_2)
data2_2 = data.frame(SampleSize, dimension,MSE2_2)
data3_2 = data.frame(SampleSize, dimension,MSE3_2)
data4_2 = data.frame(SampleSize, dimension,MSE4_2)
data5_2 = data.frame(SampleSize, dimension,MSE5_2)
data6_2 = data.frame(SampleSize, dimension,MSE6_2)

In [ ]:
# Separate into individual vectors for calculation of MSE

seq_1 <- seq(1, 1200, by = 100)
mse_table_1_2 <- rep(NA,12)
mse_table_2_2 <- rep(NA,12)
mse_table_3_2 <- rep(NA,12)
mse_table_4_2 <- rep(NA,12)
mse_table_5_2 <- rep(NA,12)
mse_table_6_2 <- rep(NA,12)

for (i in 1:length(seq_1)) {
    mse_table_1_2[i] <- mean(data1_2[seq_1[i]:(seq_1[i]+99),3])
    mse_table_2_2[i] <- mean(data2_2[seq_1[i]:(seq_1[i]+99),3])
    mse_table_3_2[i] <- mean(data3_2[seq_1[i]:(seq_1[i]+99),3])
    mse_table_4_2[i] <- mean(data4_2[seq_1[i]:(seq_1[i]+99),3])
    mse_table_5_2[i] <- mean(data5_2[seq_1[i]:(seq_1[i]+99),3])
    mse_table_6_2[i] <- mean(data6_2[seq_1[i]:(seq_1[i]+99),3])
    
}

In [ ]:
# Combine MSE results into one table 
hte_results_2 <- round(rbind(mse_table_1_2,mse_table_2_2,mse_table_3_2,mse_table_4_2,mse_table_5_2,mse_table_6_2),2)

# and then print table in LateX for export purposes
#print.xtable(xtable(head(hte_results_2)), file = "./Data2.txt")

#### Graph the results for figure in paper

In [ ]:
colnames(data1_2) <- colnames(data1)
colnames(data2_2) <- colnames(data2)
colnames(data3_2) <- colnames(data3)
colnames(data4_2) <- colnames(data4)
colnames(data5_2) <- colnames(data5)
colnames(data6_2) <- colnames(data6)

Forest=rep(c("Honest","Dishonest"),each=1200)

dataplus1 <- rbind(data1, data1_2)
dataplus1 <- cbind(dataplus1,Forest)

dataplus2 <- rbind(data2, data2_2)
dataplus2 <- cbind(dataplus2,Forest)

dataplus3 <- rbind(data3, data3_2)
dataplus3 <- cbind(dataplus3,Forest)

dataplus4 <- rbind(data4, data4_2)
dataplus4 <- cbind(dataplus4,Forest)

dataplus5 <- rbind(data5, data5_2)
dataplus5 <- cbind(dataplus5,Forest)

dataplus6 <- rbind(data6, data6_2)
dataplus6 <- cbind(dataplus6,Forest)

ggplot(dataplus1, aes(x=cut_width(SampleSize, width = 0.1), y=MSE1, fill=Forest)) + 
    geom_boxplot(color="black",outlier.color="black") + facet_grid(. ~ dimension) +
 scale_fill_brewer() +theme_light() +
 scale_x_discrete(labels= n) +
theme(legend.position="none", plot.title = element_text(size = 20), axis.text=element_text(size=16)) +
# ggtitle("Unconfounded (RCT)") +
  xlab("Number of Observations") + ylab("MSE")
options(repr.plot.width=15, repr.plot.height=6)

ggplot(dataplus2, aes(x=cut_width(SampleSize, width = 0.1), y=MSE2, fill=Forest)) + 
    geom_boxplot(color="black",outlier.color="black") + facet_grid(. ~ dimension) +
 scale_fill_brewer() +theme_light() +
 scale_x_discrete(labels= n) +
theme(plot.title = element_text(size = 20), axis.text=element_text(size=16)) +
 #ggtitle("Unconfounded (RCT) with noisy covariates") +
  xlab("Number of Observations") + ylab("MSE")
options(repr.plot.width=15, repr.plot.height=6)

ggplot(dataplus3, aes(x=cut_width(SampleSize, width = 0.1), y=MSE3, fill=Forest)) + 
    geom_boxplot(color="black",outlier.color="black") + facet_grid(. ~ dimension) +
 scale_fill_brewer() +theme_light() +
 scale_x_discrete(labels= n) +
theme(legend.position="none", plot.title = element_text(size = 20), axis.text=element_text(size=16)) +
 #ggtitle("Confounded Treatment with zero treatment effect") +
  xlab("Number of Observations") + ylab("MSE")
options(repr.plot.width=15, repr.plot.height=6)

ggplot(dataplus4, aes(x=cut_width(SampleSize, width = 0.1), y=MSE4, fill=Forest)) + 
    geom_boxplot(color="black",outlier.color="black") + facet_grid(. ~ dimension) +
 scale_fill_brewer() +theme_light() +
 scale_x_discrete(labels= n) +
theme(legend.position="none", plot.title = element_text(size = 20), axis.text=element_text(size=16)) +
# ggtitle("Confounded Treatment with Heterogeneous Treatments") +
  xlab("Number of Observations") + ylab("MSE")
options(repr.plot.width=15, repr.plot.height=6)

ggplot(dataplus5, aes(x=cut_width(SampleSize, width = 0.1), y=MSE5, fill=Forest)) + 
    geom_boxplot(color="black",outlier.color="black") + facet_grid(. ~ dimension) +
 scale_fill_brewer() +theme_light() +
 scale_x_discrete(labels= n) +
theme(legend.position="none", plot.title = element_text(size = 20), axis.text=element_text(size=16)) +
 #ggtitle("Difficult nuisance parameters, linear treatment effect function") +
  xlab("Number of Observations") + ylab("MSE")
options(repr.plot.width=15, repr.plot.height=6)

ggplot(dataplus6, aes(x=cut_width(SampleSize, width = 0.1), y=MSE6, fill=Forest)) + 
    geom_boxplot(color="black",outlier.color="black") + facet_grid(. ~ dimension) +
 scale_fill_brewer() +theme_light() +
 scale_x_discrete(labels= n) +
theme(legend.position="none", plot.title = element_text(size = 20), axis.text=element_text(size=16)) +
# ggtitle("Difficult nuisance parameters, non-linear treatment effect function") +
  xlab("Number of Observations") + ylab("MSE")
options(repr.plot.width=15, repr.plot.height=6)




----
## Part II - Empirical Applications
---

### (1) *The Impacts of Microcredit: Evidence from Bosnia and Herzegovina* (Crépon et al., 2015)

In [ ]:
# Import the cleaned dataset from STATA and restrict to those who took part in endline interviews
dataset_full <- as.data.frame(read_dta('Covariates-Bosnia-Baseline.dta'))
dataset      <- dataset_full[dataset_full$followup == 1,]

#### Estimating Average Treatment Effect on the Treated for Self-Employment Activities

In [ ]:
# Select outcome variables of interest
dep_vars <- c("bm_revenue","bm_expenses","bm_profit","inventory","y_selfempl",
              "bm_own","bm_service","bm_agric")

ate_mat <- matrix(NA, nrow = length(dep_vars), ncol = 8)

# Loop over each outcome variable 
for (i in 1:length(dep_vars)) {
    
    # Remove obs with missing outcome variable
    dataset <- dataset[!is.na(dataset[[dep_vars[i]]]),]
    
    # Covariates space, outcome Y, and treatment W
    Y <- as.matrix(dataset[,c(dep_vars[i]),drop=FALSE])  # outcome variable 
    W <- as.matrix(dataset[,c("treatment")])
    X <- as.matrix(dataset[,c("b_resp_female","b_resp_age","b_resp_age2","b_resp_ms1",
                              "b_resp_ms4","b_resp_ms5","b_resp_ss","b_resp_ul",
                              "b_kids_05","b_kids_610","b_kids_1116","b_hhmem_female", 
                              "b_hhmem_employed","b_hhmem_school","b_hhmem_retired")])
    
    df1 = data.frame(X)
    res <- cbind(df1, do.call(cbind,combn(colnames(df1), 2, 
               FUN= function(x) list(df1[x[1]]*df1[x[2]]))))
    colnames(res)[-(seq_len(ncol(df1)))] <-  combn(colnames(df1), 2, 
                 FUN = paste, collapse="_")
      
    X <- as.matrix(res)                                   
    data <- list('X' = X, 'Y' = Y, 'W' = W)

    # DML requires specific data format
    dml_data <- double_ml_data_from_data_frame(na.omit(data.table(cbind(X,W,Y))),              
                                               y_col = dep_vars[i], d_cols = "V121")  
    
    set.seed(123)      # replicability
    
    cf_honest    <- causal_forest(data$X, data$Y, data$W, honesty = TRUE,tune.parameters = "all",seed = 123)    # train the GRF Causal Forest
    
    ## DML Random Forest
    learner <- lrn("regr.ranger", num.trees=200, min.node.size=2, max.depth=5)
    g_hat <- learner$clone()                                  # use RF to estimate nuisance g() and m()
    m_hat <- learner$clone()

    DML_PLR  <- DoubleMLPLR$new(dml_data, g_hat, m_hat)   # initialize the DML Partially Linear Model 
    DML_PLR$fit()  
    params_rf    <- cbind(DML_PLR$coef,DML_PLR$se)

    ## DML LASSO
    learner <- lrn("regr.cv_glmnet", s="lambda.min",standardize=TRUE)
    g_hat <- learner$clone()                                  # use RF to estimate nuisance g() and m()
    m_hat <- learner$clone()

    DML_PLR  <- DoubleMLPLR$new(dml_data, g_hat, m_hat)   # initialize the DML Partially Linear Model 
    DML_PLR$fit()  
    params_lasso    <- cbind(DML_PLR$coef,DML_PLR$se)

    # Double Selection
    DS_c  <- summary(rlassoEffect(x = data$X, y = data$Y, d=data$W, method = "double selection"))$coefficients[1,1]
    DS_se <- summary(rlassoEffect(x = data$X, y = data$Y, d=data$W, method = "double selection"))$coefficients[1,2]
    
    ate_mat[i,1:2] <- average_treatment_effect(cf_honest)
    ate_mat[i,3:4] <- params_lasso
    ate_mat[i,5:6] <- params_rf
    ate_mat[i,7:8] <- cbind(DS_c,DS_se)
    
}

#### Print output

In [ ]:
# OLS benchmark taken directly from paper.
benchmark_ols <- matrix(c(1384, 601.4, 671.9, 0.0513, 0.0602, 0.0584, 0.0312, 0.0350,
                          981.4, 592.9, 541.3, 0.020, 0.0293, 0.031,  0.025, 0.028),
                          nrow = length(dep_vars), ncol = 2)

# combine OLS and ML estimates
results_mat <- cbind(benchmark_ols,ate_mat)


out <- cbind(as.data.frame(c(rbind(results_mat[,1],results_mat[,2]))),
      as.data.frame(c(rbind(results_mat[,3],results_mat[,4]))),
      as.data.frame(c(rbind(results_mat[,5],results_mat[,6]))),
      as.data.frame(c(rbind(results_mat[,7],results_mat[,8]))),
      as.data.frame(c(rbind(results_mat[,9],results_mat[,10]))))

# uncomment to save locally
#print.xtable(xtable(out, digits = 4), file = "./Data3.txt")

### (2) *Effect of 401(k) eligibility and participation on net financial assets*

In [ ]:
# Import the cleaned dataset from STATA 
dataset <- as.data.frame(read_dta('sipp1991.dta'))

Y <- as.matrix(dataset[,c("net_tfa"),drop=FALSE])  # outcome variable is net financial assets 
W <- as.matrix(dataset[,c("e401"), drop=FALSE])    # binary indicator for 401(k) eligibility

# Baseline variables
X_1 <- as.matrix(dataset[,c("age","inc","educ","fsize","marr","twoearn","db","pira","hown")])

# Baseline + interactions
df1 = data.frame(X_1)
res <- cbind(df1, do.call(cbind,combn(colnames(df1), 2,
             FUN= function(x) list(df1[x[1]]*df1[x[2]]))))
             colnames(res)[-(seq_len(ncol(df1)))] <-  combn(colnames(df1), 2, 
                 FUN = paste, collapse="_")
            
X_2 <- as.matrix(res)        

# Baseline + interactions + quadratics
X_3 <- as.data.frame(cbind(X_2,poly(dataset$age, 6, raw=TRUE),
                poly(dataset$inc, 8, raw=TRUE),
                poly(dataset$educ, 4, raw=TRUE),
                poly(dataset$fsize, 2, raw=TRUE)))
                                      
colnames(X_3)[46:ncol(X_3)] <- c("age_1","age_2","age_2","age_4","age_5","age_6",
                          "inc_1","inc_2","inc_3","inc_4","inc_5","inc_6","inc_7","inc_8",
                           "educ_1","educ_2","educ_3","educ_4",
                           "fsize_1","fsize_2")

X_3 <- X_3[,!names(X_3) %in% c("age_1","inc_1","educ_1","fsize_1")]
                                      
df_list <- list("X_1" = X_1,"X_2" = X_2, "X_3" = X_3)

#### Estimating Average Treatment Effect of 401(k) eligibility on Net Financial Assets

In [ ]:
ate_mat <- matrix(NA, nrow = 3, ncol = 8)
                                      
for (i in 1:3) {
    
    X <- as.matrix(df_list[[i]])
    data <- list('X' = X, 'Y' = Y, 'W' = W)
    
    # DML requires specific data format
    dml_data <- double_ml_data_from_data_frame(na.omit(data.table(cbind(X,W,Y))),              
                                               y_col = "net_tfa", d_cols = "e401")  
    
    set.seed(123)      # replicability
    
    cf_honest    <- causal_forest(data$X, data$Y, data$W, honesty = TRUE,tune.parameters = "all",seed = 123)    # train the GRF Causal Forest
    
    ## DML Random Forest
    learner <- lrn("regr.ranger", num.trees=200, min.node.size=2, max.depth=5)
    g_hat <- learner$clone()                                  # use RF to estimate nuisance g() and m()
    m_hat <- learner$clone()

    DML_PLR  <- DoubleMLPLR$new(dml_data, g_hat, m_hat)   # initialize the DML Partially Linear Model 
    DML_PLR$fit()  
    params_rf    <- cbind(DML_PLR$coef,DML_PLR$se)

    ## DML LASSO
    learner <- lrn("regr.cv_glmnet", s="lambda.min",standardize=TRUE)
    g_hat <- learner$clone()                                  # use RF to estimate nuisance g() and m()
    m_hat <- learner$clone()

    DML_PLR  <- DoubleMLPLR$new(dml_data, g_hat, m_hat)   # initialize the DML Partially Linear Model 
    DML_PLR$fit()  
    params_lasso    <- cbind(DML_PLR$coef,DML_PLR$se)

    # Double Selection
    DS_c  <- summary(rlassoEffect(x = data$X, y = data$Y, d=data$W, method = "double selection"))$coefficients[1,1]
    DS_se <- summary(rlassoEffect(x = data$X, y = data$Y, d=data$W, method = "double selection"))$coefficients[1,2]
    
    ate_mat[i,1:2] <- average_treatment_effect(cf_honest, target.sample = "all",  method = "TMLE")
    ate_mat[i,3:4] <- params_lasso
    ate_mat[i,5:6] <- params_rf
    ate_mat[i,7:8] <- cbind(DS_c,DS_se)
    
    
}

#### Print output

In [ ]:
out <- cbind(as.data.frame(c(rbind(ate_mat[,1],ate_mat[,2]))),
             as.data.frame(c(rbind(ate_mat[,3],ate_mat[,4]))),
             as.data.frame(c(rbind(ate_mat[,5],ate_mat[,6]))),
             as.data.frame(c(rbind(ate_mat[,7],ate_mat[,8]))))

# uncomment to save locally
#print.xtable(xtable(out, digits = 0), file = "./Data4.txt")

#### Treatment Heterogeneity

In [ ]:
Y <- as.matrix(dataset[,c("net_tfa"),drop=FALSE])  # outcome variable is net financial assets 
W <- as.matrix(dataset[,c("e401"), drop=FALSE])                # binary indicator 
X <- as.matrix(dataset[,c("age","inc","educ","fsize","marr","twoearn","db","pira","hown")])
data <- list('X' = X, 'Y' = Y, 'W' = W)

cf_honest <- causal_forest(data$X, data$Y, data$W, honesty = TRUE,tune.parameters = "all", seed = 123)   

In [ ]:
# Plotting the CATEs
preds <- predict(cf_honest, estimate.variance = TRUE)
tau.hat <- preds$predictions
pardef = par(mar = c(5, 4, 4, 2) + 0.5, cex.lab=1.5, cex.axis=1.5, cex.main=1.5, cex.sub=1.5)
hist(tau.hat, xlab = "estimated CATE", main = "")

# Plotting propensity score
DF = as.data.frame(cbind(X,W,tau.hat))
DF$W.hat = cf_honest$W.hat
#DF = subset(DF, DF$e401 == 0)

pardef = par(mar = c(5, 4, 4, 2) + 0.5, cex.lab=1.5, cex.axis=1.5, cex.main=1.5, cex.sub=1.5)
boxplot(DF$W.hat ~ DF$educ, data = DF, ylab = "Propensity Score", xlab = "Years of Education")
lines(smooth.spline(DF$educ, DF$W.hat), lwd = 2, col = 4)

In [ ]:
sigma.hat <- sqrt(preds$variance.estimates)
CI_95 = tau.hat + 1.96 * sigma.hat
CI_5 =  tau.hat - 1.96 * sigma.hat
  
cf_CI <- as.data.frame(cbind(tau.hat,CI_95,CI_5))
colnames(cf_CI) <- c("pred","CI_95","CI_5")

cf_melt <- reshape2::melt(cf_CI[order(cf_CI$pred),])
cf_melt$ID <- rep(1:nrow(cf_CI),times=3)

cols <- c("#000000", "#00BA38", "#0072B2")


# Plot the CATE estimates + CI for 5% and 95%
options(repr.plot.width=9, repr.plot.height=6)
ggplot(cf_melt, aes(x=ID,y=value,group=variable))+
  theme_minimal()  +ylim(0,20000) +
  geom_line(aes(color=variable,alpha=variable),size=1.1)+
  scale_alpha_manual(values=c(1.0,0.01,0.05)) +
  labs(y = "Estimated Treatment Effect ", x = "Observation") +
  theme(legend.position="none",legend.justification = 'center',plot.title = element_text(color="black", face="bold", size=15), axis.text=element_text(size=12))+ 
  scale_color_manual(values = cols) +
  geom_smooth(data=cf_melt[cf_melt["variable"] == "CI_5" | cf_melt["variable"] == "CI_95",],linetype="dashed", size=0.5)